In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('../input/feedback-prize-2021'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#pip install pymorphy2

In [ ]:
import io
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [ ]:
train_df = pd.read_csv('../input/proc-data-2/train_df.csv')
train_df

In [ ]:
train_df.shape

In [ ]:
drop_index_list = []
for i in range(len(train_df)):
    if len(train_df.loc[i].discourse_text) < 50:
        drop_index_list.append(i)
train_df.loc[drop_index_list]

In [ ]:
train_df.drop(drop_index_list, inplace=True)
train_df.shape

In [ ]:
train_df.reset_index(drop=True, inplace=True)
train_df

In [ ]:
sent_num_list = []
sent_num = 0
for i in range(0, len(train_df)):
    sent_num_list.append(sent_num)
    if i == len(train_df)-1:
        break
    if train_df.loc[i]['id'] == train_df.loc[i+1]['id']:
        sent_num += 1
    else:
        sent_num = 0
train_df['sent_num'] = sent_num_list
train_df

In [ ]:
import glob

file_name_list = []
for file in glob.glob("../input/feedback-prize-2021/test/*"):
    file_name_list.append(file[-16:])
    
file_name_list
    


In [ ]:
test_data = {}
for text in file_name_list:

    with open('../input/feedback-prize-2021/test/' + text) as f:
        contents = f.read()
        test_data[text[:-4]] = contents

In [ ]:
test_data_df = pd.DataFrame.from_dict(test_data, orient='index', columns=['discourse_text'])
test_data_df.reset_index(inplace=True)
test_data_df

In [ ]:
test_data_df

In [ ]:
import nltk

index_list = []
sentence_list = []
predictionstring = []
sent_num_list = []
for index in test_data_df['index'].values.tolist():
    sent_num = 0
    i = 0
    text = test_data_df[test_data_df['index'] == index]['discourse_text'].values[0]
    tokenized_sent = nltk.sent_tokenize(text)
    for sent in tokenized_sent:
        sent_num_list.append(sent_num)
        sent_num += 1
        string = []
        if sent != '':
            sentence_list.append(sent)
            index_list.append(index)
            for word in sent.split(' '):
                string.append(str(i))
                i += 1
            predictionstring.append(' '.join(string))
            
test_data = pd.DataFrame(index_list, columns=['id'])
test_data['discourse_text'] = sentence_list
test_data['sent_num']= sent_num_list
test_data['predictionstring'] = predictionstring
test_data['discourse_type'] = 'unknown'
test_data

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r"\w+")
proc_text = []

for sentence in test_data['discourse_text']:
    proc_sentence = tokenizer.tokenize(''.join(sentence.lower()))
    final_sentence = []
    for word in proc_sentence:
        new_word = ''.join(i for i in word if not i.isdigit())
        if new_word not in ['', 'a', 'the', 's', 'm', 't', 've']:
            final_sentence.append(new_word)
    proc_text.append(' '.join(final_sentence))


In [ ]:
test_data['proc_discourse_text'] = proc_text
test_data

In [ ]:
from gensim.models import doc2vec

def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the complaint narrative.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.TaggedDocument(v.split(), [label]))
    return labeled

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df.proc_discourse_text, train_df.discourse_type, random_state=0, test_size=0.3, stratify=train_df.discourse_type)
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
X_sub = label_sentences(test_data.proc_discourse_text, 'Sub')
all_data = X_train + X_test + X_sub
all_data

### Training the model

In [ ]:
from tqdm import tqdm
model_dbow = doc2vec.Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

In [ ]:
model_dbow = doc2vec.Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

In [ ]:
from sklearn import utils

In [ ]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

In [ ]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors

In [ ]:
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')
sub_vectors_dbow = get_vectors(model_dbow, len(X_sub), 300, 'Sub')

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(multi_class='multinomial', solver = 'sag') # solver='lbfgs' raises AttributeError: 'str' object has no attribute 'decode'
logreg.fit(train_vectors_dbow, y_train)
logreg.score(test_vectors_dbow, y_test)

In [ ]:
logreg.predict(test_vectors_dbow)

In [ ]:
sub_predict = logreg.predict(sub_vectors_dbow)
sub_predict

In [ ]:
submission = pd.DataFrame(test_data['id'], columns=['id'])
submission['class'] = sub_predict
submission['predictionstring'] = predictionstring
submission

In [ ]:
submission['class'].value_counts()

In [ ]:
submission.to_csv('submission.csv', index=False)